# AI-Assisted Programming with Langchain
> Creating full-featured apps

We've already talked about general programming, transformers, and openai. Let's see where this fits in.

# AI Ecosystem

# Langchain

[Overview of Langchain](https://python.langchain.com/docs/get_started/introduction)

## Creating and Prompting an LLM

In [ ]:
!pip install gradio langchain langchain_openai text_generation
!pip install pypdf chromadb transformers
!pip install duckduckgo-search

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

from free_access_models import ChatMistralLocal, LLMMistralLocal, MistralEmbeddingsLocal
import numpy as np

In [3]:
# auth replicated here for reference just in case you choose to do something similar
#from google.colab import userdata
#os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
#os.environ['HF_TOKEN'] = os.environ['HUGGINGFACEHUB_API_TOKEN']

# Basic Prompt Chains

See [Prompt+LLM](https://python.langchain.com/docs/expression_language/cookbook/prompt_llm_parser) for more information

In [4]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
model = ChatMistralLocal()
chain = prompt | model

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# Observe what the prompt looks like when we substitute words into it
prompt.invoke({'foo':"cats"})

ChatPromptValue(messages=[HumanMessage(content='tell me a joke about cats')])

In [6]:
# Now, actually call the entire chain on it
res = chain.invoke({'foo':"cats"})
print(res)

content=" Why don't cats play poker in the jungle?\n\nBecause there's too many cheetahs!\n\n(I hope you found that amusing!)"


## Even more simplified prompt chains

In [7]:
# Create total user prompt chain
prompt = ChatPromptTemplate.from_template("{text}")
chain = prompt | model

In [8]:
# Now, the user can submit literally whatever
res = chain.invoke({'text':"Tell me, as briefly as you can, about the pros and cons of being a pro se litigant."})
print(res)

content=" Pro se, or representing oneself in a legal proceeding without the assistance of an attorney, can be a cost-effective option for some individuals. Here are some pros and cons to consider:\n\nPros:\n1. Cost Savings: The primary advantage of being a pro se litigant is the potential for significant cost savings. Hiring an attorney can be expensive, especially in complex cases. By representing yourself, you avoid paying attorney fees.\n2. Control Over Your Case: Pro se litigants have complete control over their case, including the strategy, evidence presentation, and court appearances. This level of control can be appealing to some individuals.\n3. Learning Experience: Representing yourself in court can be an educational experience. You'll gain valuable insights into the legal system and the court process.\n\nCons:\n1. Limited Knowledge: Most people do not possess the extensive legal knowledge required to navigate complex legal proceedings effectively. This lack of expertise can l

# Creating and Using a Knowledge Base : Retrieval Augmented Generation (RAG)

* Conceptual and step-by-step guide about [RAG](https://python.langchain.com/docs/modules/data_connection/)
* Learn more about implementing [RAG](https://python.langchain.com/docs/expression_language/cookbook/retrieval)

In [9]:
from operator import itemgetter

from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from free_access_models import MistralEmbeddingsLocal

In [10]:
knowledge_base = 'Pro-Se-Handbook-APPROVED-v2019-2.pdf'

## Document Loaders and Splitters

In [11]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader(knowledge_base)

# Add the kind of text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

pages = loader.load_and_split(text_splitter=text_splitter)

In [12]:
print(f"Number of Documents: {len(pages)}\n")
pages[:2]

Number of Documents: 600



[Document(page_content='v2019 -2 Representing Y ourself in \nFederal  District Court: \n \nA Handbook for Pro Se Litigants  \n \nPRESENTED BY  \n \n \n \n \n \n \n \n \nAccess to Justice Task Force', metadata={'source': 'Pro-Se-Handbook-APPROVED-v2019-2.pdf', 'page': 0}),
 Document(page_content='Access to Justice Task Force  \nThis is an informational handbook.  This handbook is a guide for self -represented litigants. It is \nnot legal advice and should not be considered as such. Do not cite to this handbook in your filings', metadata={'source': 'Pro-Se-Handbook-APPROVED-v2019-2.pdf', 'page': 0})]

## Vector Stores: A way to store embeddings (hidden states) of your data

In [13]:
db = Chroma.from_documents(pages, MistralEmbeddingsLocal(model))

## Retrievers: How we select the most relevant data

In [14]:
# query the vector store
query = 'What forms do I need to fill out to begin with?'

# use a similarity search between the vectors
relevant_docs = db.similarity_search(query)


In [15]:
relevant_docs

[Document(page_content='When you complete the form, you will need to include the county of residence of the first', metadata={'page': 17, 'source': 'Pro-Se-Handbook-APPROVED-v2019-2.pdf'}),
 Document(page_content='If you choose to prepare your own complaint, the following information should be included.', metadata={'page': 12, 'source': 'Pro-Se-Handbook-APPROVED-v2019-2.pdf'}),
 Document(page_content='You must  explain fully the reason for your objection.  If you later learn that your', metadata={'page': 32, 'source': 'Pro-Se-Handbook-APPROVED-v2019-2.pdf'}),
 Document(page_content='Going to Governmental or Private Agencies . Consider whether there are other processes', metadata={'page': 4, 'source': 'Pro-Se-Handbook-APPROVED-v2019-2.pdf'})]

In [16]:
# or use the db as a retriever
retriever = db.as_retriever()
retrieved_docs = retriever.get_relevant_documents(query)
retrieved_docs

[Document(page_content='When you complete the form, you will need to include the county of residence of the first', metadata={'page': 17, 'source': 'Pro-Se-Handbook-APPROVED-v2019-2.pdf'}),
 Document(page_content='If you choose to prepare your own complaint, the following information should be included.', metadata={'page': 12, 'source': 'Pro-Se-Handbook-APPROVED-v2019-2.pdf'}),
 Document(page_content='You must  explain fully the reason for your objection.  If you later learn that your', metadata={'page': 32, 'source': 'Pro-Se-Handbook-APPROVED-v2019-2.pdf'}),
 Document(page_content='Going to Governmental or Private Agencies . Consider whether there are other processes', metadata={'page': 4, 'source': 'Pro-Se-Handbook-APPROVED-v2019-2.pdf'})]

## RAG
For when we want to actually do generation, but want there to be retrieved documents included

In [17]:
# Basic question answering template
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

# compose prompt
prompt = ChatPromptTemplate.from_template(template)

# retriever already created above

In [18]:
# create the chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [19]:
# get the response
res = chain.invoke(query)
res

' Based on the provided context, the documents do not directly answer the question about which forms need to be filled out to begin a process. The context discusses the importance of including certain information in forms related to preparing a complaint, objections, and considering other processes. However, it does not specify which particular forms need to be completed first.'

# Agents
Learn more about [Agents](https://python.langchain.com/docs/modules/agents/quick_start) in their Quickstart

### Choose Tools

In [20]:
# Web search tools
from langchain.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# Creating custom retriever tools
from langchain.tools.retriever import create_retriever_tool

# For tested prompts
from langchain import hub

# For agents
from langchain.agents import create_react_agent
from langchain.agents import AgentExecutor

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


In [21]:
search = DuckDuckGoSearchResults()

# example
search.run("What is the most egregious case involving of a pro se litigant in Tennessee?")

'[snippet: #5. LETTING LACK OF CONFIDENCE RULE This is similar to the previous point. In a post, Did You Come To Appease Or To Conquer? The 5 Types Of Pro Se Litigants, I discussed five types of pro se litigants. The least effective is one lacking in confidence. Many pro se litigants lose early by simply not showing up for court., title: 6 Of The Biggest Mistakes Pro Se Litigants Make - Courtroom5, link: https://courtroom5.com/blog/6-of-the-biggest-mistakes-pro-se-litigants-make/], [snippet: Posted on January 14, 2024 How are pro se litigants sanctioned in an Illinois divorce. What can pro se litigants do to avoid being sanctioned in an Illinois divorce?, title: Pro Se Litigants And Sanctions In An Illinois Divorce, link: https://rdklegal.com/pro-se-litigants-and-sanctions-in-an-illinois-divorce/], [snippet: Accordingly, most states have developed a long line of cases for the idea that "a pro se litigant is presumed to have knowledge of the law and correct legal procedures so that he r

In [22]:
# based off of the Guide we used
retriever_tool = create_retriever_tool(
    retriever,
    "pro_se_guide",
    "Use this model when the user asks for guidance or advice about being a pro se litigant. You must use this tool for advice, best practices, or guidance around being a pro se litigant!",
)

In [23]:
# define tools
tools = [search, retriever_tool]

In [24]:
# create prompt
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    HumanMessagePromptTemplate.from_template(template='Answer the following questions as best you can or reply with the most meaningful ' +
                                             'response possible. You have access to the following tools:\n\n{tools}\n\n although you can also reply conversationally when appropriate. '
                                             'Use the following format:\n\n' +
                                             'Question: the input question you must answer or statement to which you should reply\n' +
                                             'Thought: you should always think about what to do\n' +
                                             'Action: the action to take, should be one of [{tool_names}]\n'+
                                             'Action Input: the input to the action\n'+
                                             'Observation: the result of the action\n... '+
                                             '(this Thought/Action/Action Input/Observation can repeat up to 2 times)\n'+
                                             'Thought: I now know the final answer\n'+
                                             'Final Answer: the final answer to the original input question or appropriate response is\n\n'+
                                             'Begin!\n\nQuestion or message: {input}\nThought:{agent_scratchpad}'),
  ]
)

In [25]:
# Create our own llm and agent
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, run_intermediate_steps=True)

In [26]:
# Execute the agent
agent_executor.invoke(
    {"input":"What is the most recent news about pro se litigants?"}
)



> Entering new AgentExecutor chain...
 I need to find the most recent news about pro se litigants. I will use DuckDuckGo search to get the latest information.

Action: duckduckgo_results_json
Action Input: most recent news pro se litigants
[snippet: Summary Pro se litigants present unique challenges to a judicial system that prizes uniformity and consistency. The lawyers practicing in that system typically receive three years of specialized education, tracking a mostly standardized curriculum, approved by a national accrediting institution., title: The Perils of Dealing with Pro Se Litigants - American Bar Association, link: https://www.americanbar.org/groups/litigation/resources/litigation-news/2023/fall/the-perils-dealing-pro-se-litigants/], [snippet: A panel of the U.S. Court of Appeals for the Ninth Circuit, aligning with other circuits, held that pro se litigants who are also attorneys should not be afforded special consideration or be..., title: 9th Circuit Joins Other Circuits

{'input': 'What is the most recent news about pro se litigants?',
 'output': "Here are some recent news articles about pro se litigants:\n\n* [The Perils of Dealing with Pro Se Litigants - American Bar Association](https://www.americanbar.org/groups/litigation/resources/litigation-news/2023/fall/the-perils-dealing-pro-se-litigants/)\n* [9th Circuit Joins Other Circuits in Ruling Attorneys Who Are Pro Se Litigants Aren't Entitled to Special Consideration](https://www.law.com/therecorder/2023/09/08/9th-circuit-joins-other-circuits-in-ruling-attorneys-who-are-pro-se-litigants-arent-entitled-to-special-consideration/)\n* [Pros Say: New Program is Improving Pro Se Representation and Easing Court Caseloads](https://www.law.com/ctlawtribune/2023/08/31/pros-say-new-program-is-improving-pro-se-representation-and-easing-court-caseloads/)\n* [We 'live on a pro se planet'; 5th Circuit allows parents to sometimes represent children without lawyers](https://www.abajournal.com/news/article/we-live-on

## Adding the conversational component

In [27]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [28]:
# start with empty chat history. We can pass this as a parameter in the invoke
message_history = ChatMessageHistory()

# Create the agent with chat history
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    handle_parsing_errors = "Check your output and make sure it conforms, use the Action/Action Input syntax"
)

In [29]:
# Start chatting away
agent_with_chat_history.invoke(
    {"input": "I'm going pro se on a civil case against a corporation that scammed me repeatedly. "+
     "How should I start preparing? I'm in the state of Tennesse."},
    config={"configurable": {"session_id": '<something>'}},
)



> Entering new AgentExecutor chain...
 This question requires guidance on preparing for a pro se civil case in Tennessee. I will use the pro_se_guide tool for this.

Action: pro_se_guide
Action Input: Preparing for a pro se civil case in Tennessee against a corporation
has been completed .   
5. Filing  
While you may be able to file your  lawsuit in person with the Clerk’s office, most filing in

8 
  
If your case does not fall under any of these categories, you should not file it  in a federal district 
court . 
4. Which District is the proper one to file my action?

6. Prepare and file a joint written report outlining the discovery plan.

1. Have I explored alternatives to suing?  
2. Have I suffered the type of injury or harm that a Court can help me with?  
3. Does the federal district court have jurisdiction to hear my claim? I now know the initial steps for preparing a pro se civil case in Tennessee against a corporation. The first step is to determine if filing in a federal 

{'input': "I'm going pro se on a civil case against a corporation that scammed me repeatedly. How should I start preparing? I'm in the state of Tennesse.",
 'chat_history': [],
 'output': 'To prepare for a pro se civil case against a corporation in Tennessee, first determine if filing in a federal district court is appropriate. If not, file in a state court. Prepare and file a joint written report outlining the discovery plan. Consider alternatives to suing and whether the injury or harm suffered is something a court can help with.'}

In [32]:
agent_with_chat_history.invoke(
    {"input": "Thank you. What do you think will be the most difficult part of representing myself in court?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<something>"}},
)



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 The most difficult parts of representing yourself in court can include understanding complex legal procedures, preparing and filing necessary documents, presenting evidence effectively, and arguing your case persuasively.

Action: pro_se_guide
Action Input: What are some common challenges for pro se litigants in Tennessee civil cases?

Thought: Based on the guide, I understand that some common challenges for pro se litigants in Tennessee civil cases include understanding court rules and procedures, preparing and filing documents correctly, gathering and presenting evidence, and effectively arguing their case.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


meaning “for oneself.” Representing yourself in a lawsuit can be complicated, time consuming, 
and costly. Failing to follow court procedures can mean losing your case. For these reasons, you 
are urged to work with a lawyer if possible.

complex the legal issues are in the particular case and the indigent party’s ability  to investigate 
and present his or her case.  
If you are granted permission to proceed in forma pauperis , and you decide to make a

1. Have I explored alternatives to suing? 
Even if you do have the right to sue, you should carefully consider alternatives to suing. 
Lawsuits can be costly, stressful , and time -consuming. Instead of filing a lawsuit, you can try other

by the statute of limitations is a legal question which may require you to do some legal research.  You should make sure your claim is not time -barred before you file a lawsuit. 
6. Have I exhauste d all other available remedies?

...

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Based on the search results, the statute of limitations for a civil case in Tennessee varies depending on the type of claim. For example, personal injury claims have a one-year statute of limitations, while contracts have a six-year statute of limitations. It's crucial to determine which type of claim applies to your situation and check the specific statute of limitations for that claim.

Thought: Before starting the preparation for a pro se civil case against a corporation in Tennessee, it's essential to ensure that the corporation's actions constitute a valid claim and that you have not missed the statute of limitations.

Action: duckduckgo_results_json
Action Input: "how to check if a corporation is still in business in Tennessee"
[snippet: You can look up an existing business entity by completing a Tennessee business entity search. Our Tennessee Business Search guide will give you instructions for looking up all business types, including corporations and limited liability companie

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Based on the search results, to check if a corporation is still in business in Tennessee, you can visit the Tennessee Secretary of State website and conduct a business entity search using the corporation's name. This will help you determine if the corporation is still active and in good standing.

Thought: Before starting the preparation for a pro se civil case against a corporation in Tennessee, it's essential to ensure that the corporation is still in business and that you have a valid claim against them.

Thought: I now know the final answer.

Final Answer: To prepare for a pro se civil case against a corporation in Tennessee, first, ensure that you have exhausted all other available remedies and that the corporation's actions constitute a valid claim. Determine the type of claim and check the specific statute of limitations for that claim. Additionally, verify that the corporation is still in business by conducting a business entity search on the Tennessee Secretary of State websi

{'input': 'Thank you. What do you think will be the most difficult part of representing myself in court?',
 'chat_history': [HumanMessage(content="I'm going pro se on a civil case against a corporation that scammed me repeatedly. How should I start preparing? I'm in the state of Tennesse."),
  AIMessage(content='To prepare for a pro se civil case against a corporation in Tennessee, first determine if filing in a federal district court is appropriate. If not, file in a state court. Prepare and file a joint written report outlining the discovery plan. Consider alternatives to suing and whether the injury or harm suffered is something a court can help with.')],
 'output': "To prepare for a pro se civil case against a corporation in Tennessee, first, ensure that you have exhausted all other available remedies and that the corporation's actions constitute a valid claim. Determine the type of claim and check the specific statute of limitations for that claim. Additionally, verify that the co

# Bringing it all together

In [33]:
import gradio as gr

In [40]:
def ai_response(message, history):
    agent_message = {'input':message}
    response = agent_with_chat_history.invoke(agent_message, config={"configurable": {"session_id": history}})
    return response['output']

with gr.Blocks() as demo :
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(ai_response, [msg, chatbot], [msg, chatbot])

demo.launch()

Exception in thread Thread-96 (run):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 62, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 40, in _get_event_loop
    loop = events.get_event_loop_policy().get_event_loop()
  File "/usr/lib/python3.10/asyncio/events.py", line 656, in get_event_loop
    raise RuntimeError('There is no current event loop in thread %r.'
RuntimeError: There is no current event loop in thread 'Thread-96 (run)'.
/usr/lib/python3.10/threading.py:1018: RuntimeWarning: coroutine 'Server.serve' was never awaited
  self._

KeyboardInterrupt: 